In [53]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [54]:
start_date = '2019-10-30'
end_date = '2019-10-30'

#クロスデバイス売上集計スプレッドシート
gsheet_url = "https://docs.google.com/spreadsheets/d/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE/edit#gid=0"

# 全体集計（速報）

In [55]:
query = (
    f"""
    select
    concat_ws('-',year,month,day) as yymmdd
    ,ads.advertiser_id
    ,hie.advertiser_name
    ,agu1.unit
    ,ads.strategy_id
    ,hie.strategy_name
    ,seg.segment_names as segment_name
    ,ad_type
    ,case when ad_type = 0 then 'RTG'
    when ad_type = 1 then 'ATA'
    when ad_type = 2 then 'カスタム'
    when ad_type = 3 then '類似'
    when ad_type = 4 then 'DYC'
    else 'Unknown'
    end as ad_type_name
    ,ctgr.name as product_ctgr_name
    ,conc.name as product_ctgr_concrete_name
    ,sum(case when action_type = "I" then 1 else 0 end) imp
    ,sum(case when action_type = "C" then 1 else 0 end) click
    ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
    ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
    ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
    ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
    ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
    ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
    ,sum(winning_price)/1000000 as win_price
    ,sum(winning_price*(1+cast(csm.margin as float)/100))/1000000 net_sales
    ,sum(winning_price*(1+cast(csm.margin as float)/100)/(1-cast(cam.margin as float)/100))/1000000 gross_sales
    from dsplog.adserver ads 
    inner join dm.hierarchies hie on ads.target_id = hie.target_id 
    left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
    left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
    left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
    left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
    left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
    left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
    left join (select target_id,group_concat(segment_name,'|') as segment_names from dm.segment_hierarchies group by target_id) seg on ads.target_id = seg.target_id
    left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
    left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
    where 
    concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}'
    and (segment_names like '%XD%'
    or tactics_name like '%XD%')
    group by 1,2,3,4,5,6,7,8,9,10,11
    order by yymmdd
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_shukei = ir.sql_to_pandas(query)
    df_shukei = df_shukei.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
    gs = GSheet()
    sheet_index = 1
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_shukei, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s